# 3D Wind flow on the earth

In [ ]:
# conda install -c conda-forge matplotlib
# conda install -c conda-forge cartopy
# conda install -c conda-forge netcdf4

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

data = Dataset('MERRA2_300.tavg1_2d_slv_Nx.20040929.nc4',mode='r')
print(data)

## 風速算出・欠損値対応・風の流れの描画関数などを定義する

In [ ]:
def get(atr): # 属性名からデータを取得
    return data.variables[atr]

def get_uv(atr): # 属性名にUやVを追加
    return(get('U'+atr), get('V'+atr))

def fill_with_nan(x): # 欠損部分は0にする
    x_nans = x[:]; x_nans[x==x._FillValue]=0
    return x_nans

def make_ws(u, v): # 欠損部分対応や風速算出
    u_nans = fill_with_nan(u); v_nans = fill_with_nan(v)
    wind_speed = np.sqrt(u_nans**2+v_nans**2)
    return (u_nans, v_nans, wind_speed)

In [ ]:
# 最初は全球・速度表示

In [ ]:
def draw_map(u, v,        # 風ベクトル(u:経度方向, v:緯度方向)
             c,           # 色
             s,           # スケール 
             s_height,    # 人工衛星の高さ (m)
             is_draw_map, # 地図を描くか・否か
             filename):   # 画像保存するファイル名
    # 全球表示する場合
    fig = plt.figure(figsize=(40,40),facecolor='black')
    ax = plt.axes(projection=ccrs.NearsidePerspective(
          central_longitude=137.0,
          central_latitude=0,
          satellite_height=s_height, # 100000000
          false_easting=0,
          false_northing=0,
          globe=None))
    if is_draw_map:
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.LAKES)
        ax.add_feature(cfeature.RIVERS)
        ax.add_feature(cfeature.BORDERS)
        ax.coastlines(resolution="110m",linewidth=1)
        ax.gridlines(linestyle='--',color='black')
    qv = plt.quiver(lon, lat, u, v, c,
        transform=ccrs.PlateCarree(),
        scale=s, alpha = 1.0,cmap='coolwarm')
    fig.savefig(filename, format='png', dpi=120)

## 風速算出・欠損値対応・風の流れの描画関数などを定義する

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200, 'alt_in_m':2},
    #{'height':"10M",  'file':"10M.png",  'map':False, 'scale':200, 'alt_in_m':10+10},
    #{'height':"50M",  'file':"50M.png",  'map':False, 'scale':200, 'alt_in_m':50+10},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':200, 'alt_in_m':1500},      # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':200, 'alt_in_m':5500},      # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':200, 'alt_in_m':14000}      # 14,000m
]

In [ ]:
s = 2 # データ抽出の間隔
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
lon = lon[::s,::s]; lat = lat[::s,::s]
# 高さごとの風速・気温情報を可視化する
for w in winds:
    u, v = get_uv(w['height']); u, v, ws = make_ws(u, v) 
    u  = u[0,::s,::s]; v  = v[0,::s,::s]; ws = ws[0,::s,::s]
    draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])
    # 気温を可視化する場合
    #t  = get('T'+w['height']); t  = t[0,::s,::s]
    #t  = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])

## 本記事では未採用（もし画像合成をしたい場合用）

In [ ]:
from PIL import Image
margin = 200
background = Image.open("2M.png")
for i, w in enumerate(winds):
    if w['height'] != '2M':
        foreground = Image.open(w['file'])
        foreground = foreground.resize(
            (foreground.width+margin*i*2, foreground.height+margin*i*2) )
        background.putalpha(255)
        foreground.putalpha(100)
        background.paste(foreground,(-margin*i,-margin*i),foreground)
plt.imshow(background)

background.save("total.png")

## 次はメルカトルでも描いてみる

In [ ]:
def draw_map(u, v,        # 風ベクトル
             c,          # 色
             s,           # スケール 
             s_height,    # 人工衛星の高さ  
             is_draw_map, # 地図を描くか 
             filename):
    
    # メルカトル図法
    fig = plt.figure(figsize=(40,40),facecolor='black')
    ax = plt.axes(projection=ccrs.Mercator(central_longitude=135))
    # メルカトル図法(ここまで)
    
    if is_draw_map:
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.LAKES)
        ax.add_feature(cfeature.RIVERS)
        ax.add_feature(cfeature.BORDERS)
        ax.coastlines(resolution="110m",linewidth=1)
        ax.gridlines(linestyle='--',color='black')
        #plt.contourf(lon, lat, ws,
        #     transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
    qv = plt.quiver(lon, lat, 
                    u, v, 
                    c,  # 色
        transform=ccrs.PlateCarree(),
        scale=s, alpha = 1.0,cmap='coolwarm') # cmap='YlOrRd'
    fig.savefig(filename, format='png', dpi=120)

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 5 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':1000},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':1000}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':1000},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':1000}   # 14,000m
]
for w in winds:
    #u, v = get_uv(w['height'])
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    t  = get('T'+w['height'])
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]
    t  = t[0,::s,::s]
    t  = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])
    draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])

## 日本を拡大してみる、温度を使った色にする

In [ ]:
def draw_map(u, v,        # 風ベクトル
             c,          # 色
             s,           # スケール 
             s_height,    # 人工衛星の高さ  
             is_draw_map, # 地図を描くか 
             filename):

    # ミラー図法
    fig = plt.figure(figsize=(20,20),facecolor='black')
    ax = plt.axes(projection=ccrs.Miller())
    ax.set_extent([120, 150, 20, 50])
    # ミラー図法(ここまで)

    if is_draw_map:
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.LAKES)
        ax.add_feature(cfeature.RIVERS)
        ax.add_feature(cfeature.BORDERS)
        ax.coastlines(resolution="110m",linewidth=1)
        ax.gridlines(linestyle='--',color='black')
        #plt.contourf(lon, lat, ws,
        #     transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
    qv = plt.quiver(lon, lat, 
                    u, v, 
                    c,  # 色
        transform=ccrs.PlateCarree(),
        scale=s, alpha = 1.0,cmap='coolwarm') # cmap='YlOrRd'

    fig.savefig(filename, format='png', dpi=120)

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 1 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':200}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':200},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':200}   # 14,000m
]
for w in winds:
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    t  = get('T'+w['height'])
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]
    t  = t[0,::s,::s]
    t  = (t-np.min(t))/(np.max(t)-np.min(t))
    draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])
    #draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])


## Google Earthでの可視化用

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 3 # データ飛ばし(detailなら1、全体なら3)
lon = lon[::s,::s]
lat = lat[::s,::s]

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200, 'alt_in_m':2+10},
    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':200, 'alt_in_m':10+10},
    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':200, 'alt_in_m':50+10},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':200, 'alt_in_m':1500}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':200, 'alt_in_m':5500},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':200, 'alt_in_m':14000}   # 14,000m
]
for w in winds:
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]

In [ ]:
def dstLatLon(lat, lon, heading, l): 
    lat0=l/(40000*1000)*360 # 地球１周(m)/360°
    lon0=l/(40000*1000)*360/np.cos(lat)
    lat0=lat0*np.cos(heading)
    lon0=lon0*np.sin(heading)
    return lat+lat0, lon+lon0

In [ ]:
f = open('wind.kml', 'w')
f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
f.write("<kml xmlns='http://earth.google.com/kml/2.2'>\n")
f.write("<Document>\n   <name>flight</name>\n")
for k, w in enumerate(winds):
    u, v = get_uv(w['height']); u, v, ws = make_ws(u, v)
    u  = u[0,::s,::s]; v  = v[0,::s,::s]; ws = ws[0,::s,::s]
    alt = w['alt_in_m']*10 # 高度は見やすさのために10倍にしておく
    for i in range(lon.shape[0]):    # lat
        for j in range(lon.shape[1]): # lon
            #if lat[i,j]>0 or lat[i,j]<-60 or lon[i,j]>60 or lon[i,j]<-60:
            #    continue
            color = int(np.clip(10*int(ws[i,j]),0,255))
            lat_d, lon_d = dstLatLon(lat[i,j], lon[i,j],
                        np.pi/2-np.arctan2(v[i,j],u[i,j]), ws[i,j]*10000)
            f.write("<Placemark>\n        <TimeSpan>\n          <begin>"
            +'2023-10-18T00:00:00'
            +"</begin>\n        </TimeSpan>\n")        
            f.write("   <Style>\n   <LineStyle>\n")
            f.write("   <color>40"+
                   '%02x%02x%02x'%(0, 255-color, color) +"</color>\n") # 色順はABGR
            f.write("   <width>5</width>\n   </LineStyle>\n")
            f.write("   </Style>\n   <LineString>\n")
            f.write("       <extrude>0</extrude>\n")
            f.write("       <altitudeMode>absolute</altitudeMode>\n")
            f.write("        <coordinates>" 
            +str(lon[i,j])+","+str(lat[i,j])+","+str(alt)
            +" "+str(lon_d)+","+str(lat_d)+","+str(alt)+"</coordinates>\n")    
            f.write("   </LineString>\n</Placemark>\n")    
f.write("</Document></kml>\n"); f.close()

## 投影法をミラー図法表示にするためのコード

In [ ]:
def draw_map(u, v,        # 風ベクトル
             c,          # 色
             s,           # スケール 
             s_height,    # 人工衛星の高さ  
             is_draw_map, # 地図を描くか 
             filename):

# https://yyousuke.github.io/matplotlib/cartopy.html
# https://docs.wradlib.org/en/latest/notebooks/visualisation/wradlib_overlay_cartopy.html

    # 全球表示
    #fig = plt.figure(figsize=(40,40),facecolor='black')
    #ax = plt.axes(projection=ccrs.NearsidePerspective(
    #      central_longitude=137.0,
    #      central_latitude=0,
    #      #central_latitude=　38.0,
    #      satellite_height=s_height, # 100000000
    #      false_easting=0,
    #      false_northing=0,
    #      globe=None))
    
    # ミラー図法
    fig = plt.figure(figsize=(20,20),facecolor='black')
    ax = plt.axes(projection=ccrs.Miller())
    ax.set_extent([120, 150, 20, 50])
    # ミラー図法(ここまで)

    #fig = pl.figure(figsize=(10, 10))
    #ax = fig.add_subplot(111, projection=map_proj)

    # メルカトル図法
    #fig = plt.figure(figsize=(40,40),facecolor='black')
    #ax = plt.axes(projection=ccrs.Mercator(central_longitude=135))
    # メルカトル図法(ここまで)
    
    #ax.set_extent([130, 140, 30, 40], crs=ccrs.PlateCarree())

    # 周囲を黒で描く場合
    #ax.set_facecolor("black")
    
    #ax.set_global() # 全球を表示させる
    #ax.set_extent([110,150,20,60])  
    if is_draw_map:
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.LAKES)
        ax.add_feature(cfeature.RIVERS)
        ax.add_feature(cfeature.BORDERS)
        ax.coastlines(resolution="110m",linewidth=1)
        ax.gridlines(linestyle='--',color='black')
        #plt.contourf(lon, lat, ws,
        #     transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
    qv = plt.quiver(lon, lat, 
                    u, v, 
                    c,  # 色
        transform=ccrs.PlateCarree(),
        scale=s, alpha = 1.0,cmap='coolwarm') # cmap='YlOrRd'

    #plt.title('MERRA-2 Daily Average 2-meter Wind Speed, 1 June 2010', size=14)
    #cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
    #cb.set_label('m/s',size=12,rotation=0,labelpad=15)
    #cb.ax.tick_params(labelsize=10)
    
    fig.savefig(filename, format='png', dpi=120)

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 2 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]
print(lon)

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':500},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':500}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':500},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':500}   # 14,000m
]
for w in winds:
    #u, v = get_uv(w['height'])
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    t  = get('T'+w['height'])
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]
    t  = t[0,::s,::s]
    t  = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])
    draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])

## 他にもさまざまな条件で描いてみる

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 5 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]
print(lon)

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':1000},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':1000}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':1000},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':1000}   # 14,000m
]
for w in winds:
    #u, v = get_uv(w['height'])
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    t  = get('T'+w['height'])
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]
    t  = t[0,::s,::s]
    t  = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])
    draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 1 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]
print(lon)

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':True, 'scale':200}, # 850hPaは高度約1,500m
    {'height':"500",  'file':"500.png",  'map':True, 'scale':200},  # 5,500m
    {'height':"250",  'file':"250.png",  'map':True, 'scale':200}   # 14,000m
]
for w in winds:
    #u, v = get_uv(w['height'])
    u, v = get_uv(w['height'])
    u, v, ws = make_ws(u, v)
    t  = get('T'+w['height'])
    u  = u[0,::s,::s]
    v  = v[0,::s,::s]
    ws = ws[0,::s,::s]
    t  = t[0,::s,::s]
    t  = (t-np.min(t))/(np.max(t)-np.min(t))
    draw_map(u, v, t, w['scale'], 100000000000, w['map'], w['file'])
    #draw_map(u, v, ws, w['scale'], 100000000000, w['map'], w['file'])

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':False, 'scale':300},
    {'height':"500",  'file':"500.png",  'map':False, 'scale':500},
    {'height':"250",  'file':"250.png",  'map':False, 'scale':1000}
]
for w in winds:
    u, v = get_uv(w['height'])
    t = get('T'+w['height'])
    #u, v, ws = make_ws(u, v)
    u=u[0,::s,::s]
    v=v[0,::s,::s]
    #ws=ws[0,::s,::s]
    t=t[0,::s,::s]
    t = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, ws, 100, 100000000, w['map'], w['file'])
    draw_map(u, v, t, w['scale'], 100000000, w['map'], w['file'])

In [ ]:
lon, lat = np.meshgrid(get('lon'), # 経度
                       get('lat')) # 緯度
s = 2 # データ飛ばし
lon = lon[::s,::s]
lat = lat[::s,::s]

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True, 'scale':200},
#"10M""50M"は"T10M""T50M"がない
#    {'height':"10M",  'file':"10M.png",  'map':False, 'scale':50},
#    {'height':"50M",  'file':"50M.png",  'map':False, 'scale':50},
    {'height':"850",  'file':"850.png",  'map':False, 'scale':300},
    {'height':"500",  'file':"500.png",  'map':False, 'scale':500},
    {'height':"250",  'file':"250.png",  'map':False, 'scale':1000}
]
for w in winds:
    u, v = get_uv(w['height'])
    t = get('T'+w['height'])
    #u, v, ws = make_ws(u, v)
    u=u[0,::s,::s]
    v=v[0,::s,::s]
    #ws=ws[0,::s,::s]
    t=t[0,::s,::s]
    t = (t-np.min(t))/(np.max(t)-np.min(t))
    #draw_map(u, v, ws, 100, 100000000, w['map'], w['file'])
    draw_map(u, v, t, w['scale'], 2000000, w['map'], w['file'])

In [ ]:
from PIL import Image
margin = 100
background = Image.open("2M.png")
for i, w in enumerate(winds):
    if w['height'] != '2M':
        foreground = Image.open(w['file'])
        foreground = foreground.resize(
            (foreground.width+margin*i*2, foreground.height+margin*i*2) )
        background.putalpha(255)
        foreground.putalpha(100)
        background.paste(foreground,(-margin*i,-margin*i),foreground)
plt.imshow(background)

background.save("total.png")

In [ ]:
from PIL import Image

background = Image.open("MERRA2_2m_ws.png")
foreground = Image.open("MERRA2_2m_ws_.png")
foreground = foreground.resize(
    (foreground.width+50*2, foreground.height+50*2)
)

background.putalpha(255)
foreground.putalpha(128)
background.paste(foreground,(-50,-50),foreground)
#background.paste(foreground, (0, 0), foreground)
plt.imshow(background)

In [ ]:
draw_map(u, v, ws, 10000000, True, 'MERRA2_2m_ws_.png')

In [ ]:
winds = [
    {'height':"2M",   'file':"2M.png",   'map':True},
]
for w in winds:
    U, V = get_uv(w['height']) # 地上2m高さの東西方向(m/s)
    u, v, ws = fill_with_nan(U, V)
    u=u[0,::s,::s]; v=v[0,::s,::s]; ws=ws[0,::s,::s]
    draw_map(u, v, ws, 500, 100000000, w['map'], w['file'])

In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

# Set the figure size, projection, and extent
fig = plt.figure(figsize=(9,5))
ax = plt.axes(projection=ccrs.PlateCarree())

#ax.set_extent([-62,-38,35,54])  
ax.set_extent([120,150,30,50])  

ax.coastlines(resolution="50m",linewidth=1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator([130,135,140]) # 描く経度線のリスト
gl.ylocator = mticker.FixedLocator([35,40,45])         # 描く緯度線のリスト
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':10, 'color':'black'}
gl.ylabel_style = {'size':10, 'color':'black'}

# 風速(速さ)
clevs = np.arange(0,14.5,1)
plt.contourf(lon, lat, ws, clevs, transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
#plt.title('MERRA-2 2m Wind Speed and Direction, 00Z 1 June 2010', size=16)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label('m/s',size=14,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)

# 風速（ベクトル）
qv = plt.quiver(lon, lat, 
                u, 
                v,
                ws,
                scale=300, color='k')

# fig.savefig('MERRA2_2m_wsVECTORS.png', format='png', dpi=120)